In [ ]:
#import TtST_data_test.csv as teh test dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import openai

# Load the data
df_llm = pd.read_csv('TtST_llm_tabular_text.csv')

openai.api_key ='openai_api_key'




In [11]:
#perform train test split
X = df_llm.drop('Issue Lifetime', axis=1)
y = df_llm['Issue Lifetime']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#add X_train and y_train to the openai to a df_test
df_test = X_train
df_test['Issue Lifetime'] = y_train

#add X_test and y_test to the openai to a df_test
df_test = X_test
df_test['Issue Lifetime'] = y_test




In [12]:
import concurrent.futures
import tiktoken
import time


# max_token here should be one since the class (1,2,3,4) is one token long. 
def query_chatgpt(prompt, model, temperature=0.0,  max_tokens=1, max_retries=5):
    """
    Function to query ChatGPT-4 with a given prompt, with retries for timeouts.

    :param prompt: Prompt string to send to ChatGPT-2.5
    :param model: The model to use, default is ChatGPT-3.5
    :param max_tokens: Maximum number of tokens to generate
    :param max_retries: Maximum number of retries for timeout
    :return: Response from ChatGPT-4o-mini or None if all retries fail
    """
    attempt = 0
    max_content_tokens = 3999
    encoding = tiktoken.get_encoding("cl100k_base")
    encoding = tiktoken.encoding_for_model("gpt-4o-mini-2024-07-18")

    # Function to truncate the message and avoid passing the limit of 4k tokens per gpt-3.5 fine-tuned model limitations
    def truncate_message(message, max_length):
        tokens = encoding.encode(message)
        if len(tokens) > max_length:
            truncated_tokens = tokens[:max_length]
            message = encoding.decode(truncated_tokens)
        return message

    # Truncate the prompt if necessary
    prompt = truncate_message(prompt, max_content_tokens)

    while attempt < max_retries:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(
                openai.chat.completions.create,
                model=model,
                messages=[{"role": "system", "content": "GitHub Issue Lifetime Classifier"}, {"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature
            )
            try:
                response = future.result(timeout=5)  # 5 seconds timeout
                return response.choices[0].message.content
            except concurrent.futures.TimeoutError:
                print(f"Attempt {attempt + 1}/{max_retries} - Request timed out. Retrying...")
            except Exception as e:
                print(f"Attempt {attempt + 1}/{max_retries} - An error occurred: {e}")
            finally:
                attempt += 1

    print("Failed to get a response after several retries.")
    return None
    
labels =[ 1, 2, 3, 4, 5]

In [13]:
y_true = []
y_pred = []

y_llm = []

iterations = len(df_test)
#lifetime = df_test['Issue Lifetime']
#predictors = df_test.drop('Issue Lifetime', axis=1)

# Generate the feature string dynamically
def generate_prompt(row):
    feature_str = "\n".join([f"- {col}: {row[col]}" for col in row.index if col != "Issue Lifetime"])
    
    prompt = (
        "Classify, using ONLY 1 NUMBER between 1 and 5, the following GitHub issue, where:\n"
        "1 = 'Less than 1 hour'\n"
        "2 = 'Between 1 hour and 1 day'\n"
        "3 = 'Between 1 day and a week'\n"
        "4 = 'Between 1 week and 1 month'\n"
        "5 = 'More than a month'\n"
        f"Based on its features:\n{feature_str}"
    )
    
    return prompt

# Now let's loop through the test data and classify the GitHub issues
for i, row in df_test.iterrows():
    correct_label = int(row['Issue Lifetime'])  # Access the correct label directly from the row
    description = row.drop('Issue Lifetime').to_string()  # Get the row data as a string

    print(f"Correct PR type: {correct_label}")
    
    # Generate the prompt
    prompt = generate_prompt(row)
    
    response = query_chatgpt(prompt, model='gpt-4o-mini-2024-07-18')
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = int(response)
    print(f"Predicted PR Lifetime: {predicted_label}")
    
    if predicted_label == correct_label:
        y_llm.append(1)
    else:
        y_llm.append(0)
        
    # Append to lists for evaluation
    y_true.append(correct_label)
    y_pred.append(predicted_label)
    time.sleep(3)  # Wait for 3 seconds before retrying

Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 2
Predicted PR Lifetime: 1
Correct PR type: 2
Predicted PR Lifetime: 2
Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 2
Predicted PR Lifetime: 3
Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 2
Predicted PR Lifetime: 4
Correct PR type: 2
Predicted PR Lifetime: 3
Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 3
Predicted PR Lifetime: 1
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 3
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 2
Predicted PR Lifetime: 1
Correct PR type: 3
Predicted PR Lifetime: 1
Correct PR type: 3
Predicted PR Lifetime: 1
Correct PR type: 2
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR 

In [16]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score


# Calculate weighted average F1-score, precision, and recall
f1 = f1_score(y_true, y_pred, labels=labels, average='weighted')
precision = precision_score(y_true, y_pred, labels=labels, average='weighted')
recall = recall_score(y_true, y_pred, labels=labels, average='weighted')

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=labels)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results = {}
for i, label in enumerate(labels):
    results[label] = {'TP': cm[i, i]}
    results[label]['FP'] = cm[:, i].sum() - cm[i, i]
    results[label]['FN'] = cm[i, :].sum() - cm[i, i]
    results[label]['TN'] = cm.sum() - (results[label]['TP'] + results[label]['FP'] + results[label]['FN'])

# Print results_fb
for label, metrics in results.items():
    print(f"{label}: {metrics}")

# Save results_fb to CSV
results_df = pd.DataFrame(results).T
results_df['F1-score'] = f1
results_df['Recall'] = recall
results_df['Precision'] = precision

#results_df.to_csv('confusion_matrix.csv')

print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"F1-score = {f1}")

1: {'TP': np.int64(33), 'FP': np.int64(110), 'FN': np.int64(421), 'TN': np.int64(1070)}
2: {'TP': np.int64(532), 'FP': np.int64(652), 'FN': np.int64(209), 'TN': np.int64(241)}
3: {'TP': np.int64(49), 'FP': np.int64(245), 'FN': np.int64(283), 'TN': np.int64(1057)}
4: {'TP': np.int64(3), 'FP': np.int64(10), 'FN': np.int64(104), 'TN': np.int64(1517)}
5: {'TP': np.int64(0), 'FP': np.int64(0), 'FN': np.int64(0), 'TN': np.int64(1634)}
Precision = 0.31685691316964265
Recall = 0.37760097919216645
F1-score = 0.3164542868784254


/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy = {accuracy}")

Accuracy = 0.37760097919216645
